# Testing loading and saving with keras

## Goals

- [x] test a model with custom layer, save it to file and load it back successfully
- [x] stack two models together after loading from file


## Open questions

- TODO - how to compute bottlenecks ? - i.e. cache the predictions
- TOTO - Why is the m.trainable = False not working with the sequential model?
- DONE - how to most efficiently store custom layers?
  - Have a layer zoo with code for each. filename = layer name
- DONE - in stacking multiple functional models, how to retrieve the input feature?
  - use model.input


--------------------

## Loading the data

In [1]:
import keras

Using Theano backend.
ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.


In [2]:
from keras.layers import Input, Dense, Convolution2D, Activation, Flatten
from keras.models import Model
from keras.layers.pooling import MaxPooling2D
from keras.utils.np_utils import to_categorical


from keras.datasets import cifar10

(X_train, y_train_raw), (X_test, y_test_raw) = cifar10.load_data()

## TODO - one-hot encode
y_train = to_categorical(y_train_raw)
y_test = to_categorical(y_test_raw)

print(X_train.shape)
print(X_test.shape)
y_train.shape


(50000, 3, 32, 32)
(10000, 3, 32, 32)


(50000, 10)

## Build model with a custom layer

In [3]:
## define my own layer

from keras import backend as K
from keras.engine.topology import Layer
import numpy as np

class MyLayer(Layer):
    def __name__(self):
        return "MyLayer"
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.W = self.add_weight(shape=(input_shape[1], self.output_dim),
                                 initializer='uniform',
                                 trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, x, mask=None):
        return K.dot(x, self.W)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], self.output_dim)
    
    def get_config(self):
        config = {'output_dim': self.output_dim}
        base_config = super(MyLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [4]:
## Build the model

inputs = Input(shape=X_train.shape[1:])
## Conv
x = Convolution2D(nb_filter = 16, nb_row = 3, nb_col=3, activation='relu')(inputs)
## Max pool
x = MaxPooling2D(strides = (2,2))(x)
## Conv
x = Convolution2D(nb_filter = 16, nb_row = 3, nb_col=3, activation='relu')(x)
## Max pool
x = MaxPooling2D(strides = (2,2))(x)
x = Flatten()(x)

x = MyLayer(output_dim = 10)(x)
## pooling

x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(y_train.shape[1], activation='softmax')(x)

# this creates a model that includes
# the Input layer and three Dense layers
model = Model(input=inputs, output=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
## Fit the model
model.fit(X_train, y_train, batch_size = 128, nb_epoch = 1)  # starts training

Epoch 1/1
50000/50000 [==============================] - 107s - loss: 1.8901 - acc: 0.3308   


## Save and load the model

In [6]:
## save the model to hdf5
model.save("test2.h5")

ImportError: No module named 'h5py'

In [431]:
## read the model from file
from keras.models import load_model

m = load_model("test2.h5", custom_objects={"MyLayer": MyLayer})

In [319]:
## Make models predictions
np.std(m.predict(X_test)- y_test)

0.26666272198223717

In [320]:
## Save and load to config
config = model.get_config()
model = Model.from_config(config, custom_objects= {"MyLayer": MyLayer})


In [321]:
## Read, write from json
from keras.models import model_from_json, load_model
json_str = model.to_json()
m = model_from_json(json_str, custom_objects = {"MyLayer": MyLayer})

In [322]:
## my layer information
from keras.utils.layer_utils import layer_from_config
c = m.layers[6].get_config()
print(c)

## TODO - fix this error 
##l = layer_from_config(c, custom_objects={"MyLayer": MyLayer})


{'name': 'mylayer_12', 'output_dim': 10, 'trainable': True}


In [414]:
m.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_39 (InputLayer)            (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
convolution2d_25 (Convolution2D) (None, 30, 30, 16)    448         input_39[0][0]                   
____________________________________________________________________________________________________
maxpooling2d_25 (MaxPooling2D)   (None, 15, 15, 16)    0           convolution2d_25[0][0]           
____________________________________________________________________________________________________
convolution2d_26 (Convolution2D) (None, 13, 13, 16)    2320        maxpooling2d_25[0][0]            
___________________________________________________________________________________________

## Stack two models after loading from the data

Setup:
- load the conv from model
- given 10 classes, predict some other thing

In [336]:
# y_train_final = np.random.normal(size = (y_train.shape[0], 1))
# y_test_final = np.random.normal(size = (y_test.shape[0], 1))

y_train_final = y_train_raw
y_test_final = y_test_raw


In [337]:
y_train_final.shape

(50000, 1)

In [425]:
## Specify a different model
inp = Input(shape=y_train.shape[1:])

predictions = Dense(output_dim = y_train_final.shape[1])(inp)

# this creates a model that includes
# the Input layer and three Dense layers
model = Model(input=inp, output=predictions)
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

In [426]:
model.fit(y_train, y_train_final, nb_epoch = 1)

Epoch 1/1
50000/50000 [==============================] - 2s - loss: 18.1199 - mean_squared_error: 18.1199     


In [427]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_44 (InputLayer)            (None, 10)            0                                            
____________________________________________________________________________________________________
dense_55 (Dense)                 (None, 1)             11          input_44[0][0]                   
Total params: 11
Trainable params: 11
Non-trainable params: 0
____________________________________________________________________________________________________


# Make a final model:

## take the model and make the output as input

In [432]:
from keras.models import Sequential



## TODO - how to get Input(,).. for the inputs model ?
#input_layer = Input(shape = m.input_shape[1:])
input_layer = m.input
m.trainable = False
o = m(input_layer)
pred = model(o)
ms = Model(input = input_layer, output = pred)

# # Combining the models
# ms = Sequential()
# ## Disable training of m
# ms.add(m)
# ms.add(model)
# m.trainable = False


In [433]:
m.input_shape[1:]

(32, 32, 3)

In [434]:
## Weights before
ms.layers[1].get_weights()

[array([[[[  1.62795894e-02,  -1.10522009e-01,   1.08729452e-01,
            -5.29595912e-02,   9.13434699e-02,  -5.01305051e-02,
             7.80148134e-02,  -2.10225508e-02,   1.81074485e-01,
            -9.65163186e-02,   7.34261572e-02,   1.27381682e-01,
            -1.43708691e-01,   1.55294240e-01,  -4.48099449e-02,
            -1.16600528e-01],
          [ -2.72225738e-02,  -9.25690755e-02,  -1.70763105e-01,
            -1.19252533e-01,   4.71934527e-02,   2.70957928e-02,
            -8.37612972e-02,   1.60339698e-01,  -8.14339370e-02,
            -1.20209210e-01,   3.93146761e-02,   1.70531357e-03,
             6.95383251e-02,  -1.18462265e-01,  -5.72150908e-02,
            -8.25282633e-02],
          [  4.07722630e-02,  -6.28030077e-02,  -6.00527925e-03,
            -8.88971761e-02,   1.42754748e-01,  -9.88525450e-02,
            -1.12998828e-01,  -2.15416953e-01,   5.97720295e-02,
             1.98679104e-01,   1.74118385e-01,   1.45688251e-01,
            -1.50285393e-01,  

In [435]:
ms.summary() ## dense_49 refers to layer in model_26s

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_39 (InputLayer)            (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
model_31 (Model)                 (None, 10)            14042       input_39[0][0]                   
____________________________________________________________________________________________________
model_32 (Model)                 (None, 1)             11          model_31[1][0]                   
Total params: 14,053
Trainable params: 14,053
Non-trainable params: 0
____________________________________________________________________________________________________


In [436]:
ms.compile(optimizer='adam',
           loss='mse',
           metrics=['mse'])

In [437]:
ms.fit(X_train, y_train_final, nb_epoch=1)

Epoch 1/1
50000/50000 [==============================] - 12s - loss: 9.5740 - mean_squared_error: 9.5740    


In [438]:
## Weights after
ms.layers[1].get_weights()

[array([[[[  1.62795894e-02,  -1.10522009e-01,   1.08729452e-01,
            -5.29595912e-02,   9.13434699e-02,  -5.01305051e-02,
             7.80148134e-02,  -2.10225508e-02,   1.81074485e-01,
            -9.65163186e-02,   7.34261572e-02,   1.27381682e-01,
            -1.43708691e-01,   1.55294240e-01,  -4.48099449e-02,
            -1.16600528e-01],
          [ -2.72225738e-02,  -9.25690755e-02,  -1.70763105e-01,
            -1.19252533e-01,   4.71934527e-02,   2.70957928e-02,
            -8.37612972e-02,   1.60339698e-01,  -8.14339370e-02,
            -1.20209210e-01,   3.93146761e-02,   1.70531357e-03,
             6.95383251e-02,  -1.18462265e-01,  -5.72150908e-02,
            -8.25282633e-02],
          [  4.07722630e-02,  -6.28030077e-02,  -6.00527925e-03,
            -8.88971761e-02,   1.42754748e-01,  -9.88525450e-02,
            -1.12998828e-01,  -2.15416953e-01,   5.97720295e-02,
             1.98679104e-01,   1.74118385e-01,   1.45688251e-01,
            -1.50285393e-01,  

### Conclusions

- Weights indeed don't change when we use the functional input.
- I wasn't able to get it work with the Sequential layers